In [1]:
##################################
### Import the usual libraries ###
##################################

### Other
import warnings
warnings.filterwarnings('ignore')

### Matplotlib
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams.update({'font.size': 30})

In [ ]:
# def MCDistribution(self,teff,teff_err,feh,feh_err,am,am_err,mag,mag_err,number):
#         '''
#         Generate a distribution to do a Monte Simulation to calculate age and extinction
            
#         Inputs:
#         ------
            
#             teff: temperature of star
#             teff_err: error in temperature of star
#             feh: metallicity of star
#             feh_err: error in metallicity of star
#             am: alpha abundance of star
#             am_err: error in alpha abundance of star
#             mag: magnitude of star corresponding to self.band 
#             mag_err: error in magnitude of star corresponding to self.band
#             number: number of points to sample
              
#         Outputs:
#         -------
#             noisy_feh
#             noisy_teff
#             noisy_mag
#         '''
            
#         noisy_teff = np.random.normal(teff,teff_err,number)
#         noisy_feh = np.random.normal(feh,feh_err,number)
#         noisy_am = np.random.normal(am,am_err,number)
#         noisy_mag = np.random.normal(mag,mag_err,number)
            
#         return noisy_teff, noisy_feh, noisy_am, noisy_mag

In [36]:
import numpy as np
import astropy
from astropy.io import fits
from astropy.table import Table
from scipy.interpolate import InterpolatedUnivariateSpline

def closest(data,value):
    '''
    Find nearest value in array to given value
        
    Inputs:
    ------
        data: data to search through 
        value: value of interest
        
    Output:
    ------
        close_value
    '''
    
    value = np.asarray(value)
    data = np.asarray(data)
    
    close_value = data[(np.abs(np.subtract(data,value))).argmin()]
    return close_value

class WhatsMyAgeAgain():
    '''
    Class to calculate a star's age and extinction using PARSEC isochrones
    '''
    def __init__(self,band,distance,isochrones):
        
        '''
        #mag: [float] apparent magnitude of star
        #feh: [float] metallicity of star
        band: [str] photometric band of interest
        distance: [float] distance to star in pc
        isochrones: [astropy.io.fits.fitsrec.FITS_rec / astropy.table.table.Table] PARSEC isochrone table
        '''
        
        # stellar parameters
#         self.mag = mag
#         self.feh = feh
        self.dist = distance
        
        # PARSEC isochrones
        self.band = band
        self.iso = isochrones
        
        # effective wavelengths of different bands
#         self.leff = {'BP':0.5387,'G':0.6419,'RP':0.7667,'J':1.2345,'H':1.6393,'K':2.1757}

    def Salaris(self,feh,am):
        '''
        Calculate the Salaris et al. 1993 corrected [Fe/H]
        
        Inputs:
        ------
            feh: uncorrected metallicity of star
            am: alpha abundance of star
        
        Output:
        ------
            sal_feh: corrected metallicity
        '''
        
        sal_feh = feh + np.log10(0.638*(10**am)+0.362)
        return sal_feh
        
    def Teff2AppMag(self,teff_feh,age,al):
        '''
        Calculate the expected apparent magnitude of a star
        
        Inputs:
        ------
            XXXteff: temperature of star
            XXXfeh: metallicity of star
            teff_feh: (teff,feh) temperature and metallicity of star
            age: age of star
            al: extinction in the same band used to initialize the class
        
        Output:
        ------
            calc_mag: expected intrinsic magnitude for the given temperature
        '''
        
        teff = teff_feh[0]#[:,0]
        feh = teff_feh[1]#[:,1]
        single = self.iso[np.where((self.iso['logAge']==closest(self.iso['logAge'],np.log10(age*10**9)))&
                                   (self.iso['MH']==closest(self.iso['MH'],np.mean(feh))))]
    
        sidx = np.argsort(single['logTe'])
        slogTe = single['logTe'][sidx]
        smag = single[self.band][sidx]

        _, uidx = np.unique(slogTe,return_index=True)
    
        try:
            spl = InterpolatedUnivariateSpline(slogTe[uidx],smag[uidx])
    
            calc_mag = spl(np.log10(teff))+5.0*np.log10(self.dist)-5.0+al
            return calc_mag
    
        except:
            return -9999.0
        
        

In [3]:
# massive
massive = fits.getdata('/Users/joshuapovick/Desktop/Research/parsec/parsec_massive.fits.gz',0)
massive = massive[np.where(massive['label']==3.0)]

In [6]:
### GCS Data
gcs = fits.getdata('/Users/joshuapovick/Desktop/Research/fits/allStar-r13-l33-58932beta_apa_dist_galvel_gc.fits.gz')
cln = np.where((gcs['FE_H']>-9999.0)&(gcs['AK_TARG']>-9999.0)&(gcs['LOGG']>0.0)&(gcs['M_H_ERR']>-90.0)&
                (gcs['C_FE']>-9999.0)&(gcs['N_FE']>-9999.0))
gcs = Table(gcs[cln])

### Find Cluster with more than one star

idx = []
for i in range(len(np.unique(gcs['CLUSTER']))):
    idx.append(np.squeeze(np.where(gcs['CLUSTER']==np.unique(gcs['CLUSTER'])[i])))

for i in range(len(idx)):
    try:
        len(idx[i])
    except:
        print('bad: ',i)
        
good_names = []
for i in np.asarray(idx)[np.delete(np.asarray(list(range(len(idx)))),[24,27,37])]:
    if len(i)>10:
        good_names.append(gcs['CLUSTER'][i][0])

print(good_names)

good_clus = np.where((gcs['CLUSTER']=='47Tuc')|(gcs['CLUSTER']=='M10')|(gcs['CLUSTER']=='M107')|
                     (gcs['CLUSTER']=='M12')|(gcs['CLUSTER']=='M13')|(gcs['CLUSTER']=='M19')|
                     (gcs['CLUSTER']=='M2')|(gcs['CLUSTER']=='M22')|(gcs['CLUSTER']=='M3')|
                     (gcs['CLUSTER']=='M4')|(gcs['CLUSTER']=='M5')|(gcs['CLUSTER']=='M53')|
                     (gcs['CLUSTER']=='M54')|(gcs['CLUSTER']=='M55')|(gcs['CLUSTER']=='M71')|
                     (gcs['CLUSTER']=='M79')|(gcs['CLUSTER']=='NGC1851')|(gcs['CLUSTER']=='NGC2808')|
                     (gcs['CLUSTER']=='NGC288')|(gcs['CLUSTER']=='NGC3201')|(gcs['CLUSTER']=='NGC362')|
                     (gcs['CLUSTER']=='NGGC6388')|(gcs['CLUSTER']=='NGC6397')|(gcs['CLUSTER']=='NGC6752')|
                     (gcs['CLUSTER']=='omegaCen'))

gcs = gcs[good_clus]

bad:  24
bad:  27
bad:  37
['47Tuc', 'M10', 'M107', 'M12', 'M13', 'M19', 'M2', 'M22', 'M3', 'M4', 'M5', 'M53', 'M54', 'M55', 'M71', 'M79', 'NGC1851', 'NGC2808', 'NGC288', 'NGC3201', 'NGC362', 'NGC6388', 'NGC6397', 'NGC6752', 'omegaCen']


In [7]:
def closest2(data,value):
    '''
    Find values of two elements closest to the given value
    
    Inputs:
    ------
        data: data to search through 
        value: value of interest
        
    Output:
    ------
        close1: closest value under the given value
        close2: closest value over the given value
    '''
    
    value = np.asarray(value)
    data = np.asarray(data)
    
    close1 = data[(np.abs(np.subtract(data,value))).argmin()]
    
    data = data[np.where(data!=close1)]
    
    close2 = data[(np.abs(np.subtract(data,value))).argmin()]
    
    return close1,close2
    

In [9]:
closest2([9,202,93,9,92929,4,5,3,7,43,450,1000,5000],4000)

(5000, 1000)

In [12]:
from scipy.optimize import minimize

In [25]:
g_idx = 1000
CalcAgek = WhatsMyAgeAgain('Ksmag',10400,massive)

feh = CalcAgek.Salaris(gcs['M_H'][g_idx],gcs['ALPHA_M'][g_idx])

minimize(CalcAgek.chi_square,[gcs['TEFF'][g_idx],gcs['K'][g_idx],feh])

TypeError: chi_square() missing 2 required positional arguments: 'age' and 'al'

In [26]:
from tqdm import tqdm_notebook

age_len = len(np.unique(massive['logAge']))
al_len = len(np.arange(0.0,1.0,0.1))
chi_space = np.empty((age_len,al_len))
for i in tqdm_notebook(range(len(np.unique(massive['logAge'])))):
    for j in range(len(np.arange(0.0,1.0,0.1))):
        chi_space[i][j] = CalcAgek.chi_square([gcs['TEFF'][g_idx],gcs['K'][g_idx],feh],
                                              10**np.unique(massive['logAge'])[i]/10**9,
                                              np.arange(0.0,1.0,0.1)[j])

In [38]:
max(closest(chi_space/((gcs['FE_H_ERR'][g_idx]**2+gcs['TEFF_ERR'][g_idx]**2+gcs['K_ERR'][g_idx]**2)*(2)),1))

8.825732095091106e-05

In [39]:
np.where(chi_space/((gcs['FE_H_ERR'][g_idx]**2+gcs['TEFF_ERR'][g_idx]**2+gcs['K_ERR'][g_idx]**2)*(2))==max(closest(chi_space/((gcs['FE_H_ERR'][g_idx]**2+gcs['TEFF_ERR'][g_idx]**2+gcs['K_ERR'][g_idx]**2)*(2)),1)))

(array([519]), array([9]))

In [31]:
10**np.unique(massive['logAge'])[278]/10**9

6.974932354244468

In [40]:
10**np.unique(massive['logAge'])[519]/10**9

12.999899653754525

In [41]:
np.arange(0.0,1.0,0.1)[9]

0.9

In [43]:
np.mean([6.974932354244468,12.999899653754525])

9.987416003999497